In [1]:
import torch
print(torch.nn.__dict__)

{'__name__': 'torch.nn', '__doc__': None, '__package__': 'torch.nn', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7f1c443d7110>, '__spec__': ModuleSpec(name='torch.nn', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f1c443d7110>, origin='/home/hosam.elgendy/miniconda3/envs/test/lib/python3.12/site-packages/torch/nn/__init__.py', submodule_search_locations=['/home/hosam.elgendy/miniconda3/envs/test/lib/python3.12/site-packages/torch/nn']), '__path__': ['/home/hosam.elgendy/miniconda3/envs/test/lib/python3.12/site-packages/torch/nn'], '__file__': '/home/hosam.elgendy/miniconda3/envs/test/lib/python3.12/site-packages/torch/nn/__init__.py', '__cached__': '/home/hosam.elgendy/miniconda3/envs/test/lib/python3.12/site-packages/torch/nn/__pycache__/__init__.cpython-312.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifier

In [11]:
import re

file_path = "/home/yasser.attia/hosam.elgendy/GeoLLaVA/results_prune_LLaVA-NeXT-Video-7B-hf_sample_QLORA_4bit_r64_alpha128.json"


def fix_json_formatting(file_path):
    # Read the JSON file as a string
    with open(file_path, 'r') as file:
        json_string = file.read()

    # Fix missing commas between objects in the array
    fixed_json_string = re.sub(r'\}\s*\{', '},{', json_string)

    # Save the fixed JSON back to a new file
    # fixed_file_path = file_path.replace(".json", "_fixed.json")
    fixed_file_path = file_path
    with open(fixed_file_path, 'w') as fixed_file:
        fixed_file.write(fixed_json_string)

    print(f"Fixed JSON saved to: {fixed_file_path}")

#Run the formatting code
fix_json_formatting(file_path)


Fixed JSON saved to: /home/yasser.attia/hosam.elgendy/GeoLLaVA/results_prune_LLaVA-NeXT-Video-7B-hf_sample_QLORA_4bit_r64_alpha128.json


In [12]:
import json


# Function to clean the "generated" field
def clean_generated_field(data):
    for item in data:
        if "generated" in item:
            generated_text = item["generated"]
            # Look for the phrase "ASSISTANT: Answer:" and clean the text before and including it
            keyword = "Answer:"
            if keyword in generated_text:
                # Keep everything after "ASSISTANT: Answer:"
                cleaned_text = generated_text.split(keyword)[1]  # Only keep what's after the keyword
                item["generated"] = cleaned_text.strip()
    return data

# Load JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Clean the generated fields
cleaned_data = clean_generated_field(data)

# Save the cleaned data back to a file
with open(file_path, 'w') as file:
    json.dump(cleaned_data, file, indent=4)

print("Cleaning completed and output saved")


Cleaning completed and output saved


In [13]:
import json
import pandas as pd
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util
from multiprocessing import Pool
import torch
import json
import pandas as pd


file_format = file_path.split('/')[-1].replace(".json", ".csv")

with open(file_path, 'r') as f:
    json_data = json.load(f)
# Initialize the metrics: Rouge, Bleu, and BERT
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
bert_model = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda' if torch.cuda.is_available() else 'cpu')

# Function to calculate metrics for a single sample
def compute_metrics(sample):
    true_desc = sample["true"]
    gen_desc = sample["generated"]

    # ROUGE Scores
    rouge_scores = scorer.score(true_desc, gen_desc)

    # Limit BLEU to 2-gram (unigram and bigram) matches with smoothing
    smoothing_function = SmoothingFunction().method1
    bleu_score = sentence_bleu([true_desc.split()], gen_desc.split(), weights=(0.5, 0.5), smoothing_function=smoothing_function)

    return {
        "id": sample["id"],
        "video": sample["video"],
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "bleu": bleu_score
    }

# Batch process BERT embeddings for similarity
def compute_bert_similarity(true_descriptions, generated_descriptions):
    true_embs = bert_model.encode(true_descriptions, convert_to_tensor=True, batch_size=512)
    gen_embs = bert_model.encode(generated_descriptions, convert_to_tensor=True, batch_size=512)

    # Calculate cosine similarities in a batched manner
    bert_similarities = [util.pytorch_cos_sim(true_emb, gen_emb).item() for true_emb, gen_emb in zip(true_embs, gen_embs)]
    return bert_similarities


# Extract true and generated descriptions for BERT similarity in batches
true_descriptions = [sample['true'] for sample in json_data]
generated_descriptions = [sample['generated'] for sample in json_data]

# Compute BERT similarity in batch
bert_similarities = compute_bert_similarity(true_descriptions, generated_descriptions)

# Compute ROUGE and BLEU in parallel using multiprocessing
with Pool() as pool:
    metric_results = pool.map(compute_metrics, json_data)

# Combine results
for i, result in enumerate(metric_results):
    result['bert_similarity'] = bert_similarities[i]

# Convert results to DataFrame
df = pd.DataFrame(metric_results)

# Calculate averages for numeric columns
avg_results = df.mean(numeric_only=True)

# Add the average row to the DataFrame
avg_results_row = pd.DataFrame(avg_results).transpose()
avg_results_row['id'] = 'average'
avg_results_row['video'] = 'average'

# Append the average row to the original DataFrame
df_with_avg = pd.concat([df, avg_results_row], ignore_index=True)

# Save to CSV with the average row included
df_with_avg.to_csv(file_format, index=False)



/home/yasser.attia/miniconda3/envs/test/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly